# 0.0 IMPORTS

## 0.1 Libraries

In [1]:
#from pipeline.Pipeline import churn_pipeline
import pandas as pd
import joblib as jb
import json
import requests

## 0.3 Load Data

In [2]:
df_test_dataset = pd.read_feather('00-Data/test_dataset.feather')

In [3]:
# convert Dataframe to json
data = json.dumps(df_test_dataset.to_dict(orient='records'))

In [4]:
# API Call
url = ' http://localhost:5000/churn/predict'
#url = ''
header = {'Content-type': 'application/json'} 
data = data

r = requests.post(url, data=data, headers=header)
print('Status Code {}'.format(r.status_code))

Status Code 200


In [5]:
d1 = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [6]:
d1

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,yhat_proba_level_02,final_result
0,15749540,Hsiung,585,France,Male,36,7,0.00,2,1,0,94283.09,0.341356,0
1,15807340,O'Donnell,525,Germany,Male,33,4,131023.76,2,0,0,55072.93,0.598406,1
2,15801062,Matthews,557,Spain,Female,40,4,0.00,2,0,1,105433.53,0.337237,0
3,15572801,Krischock,639,Spain,Male,34,5,139393.19,2,0,0,33950.08,0.245360,0
4,15600708,Calabresi,640,Spain,Female,34,3,77826.80,1,1,1,168544.85,0.549988,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,15729083,Gorman,674,France,Male,36,2,154525.70,1,0,1,27468.72,0.433002,0
1996,15625881,Koehler,634,Germany,Male,37,3,111432.77,2,1,1,167032.49,0.439162,0
1997,15724321,Baresi,516,Germany,Female,47,9,128298.74,1,0,0,149614.17,0.711001,1
1998,15685109,Teng,689,France,Male,39,7,0.00,2,0,0,14917.09,0.246650,0


In [7]:
from sklearn.metrics         import (roc_auc_score, average_precision_score, precision_score,
                                    recall_score, f1_score, cohen_kappa_score, balanced_accuracy_score)

def ml_scores(model_name, y, yhat, yhat_proba):
    precision = precision_score(y, yhat)
    recall = recall_score(y, yhat)
    f1 = f1_score(y, yhat)
    kappa = cohen_kappa_score(y, yhat)
    balanced_accuracy = balanced_accuracy_score(y, yhat)
    ap = average_precision_score(y, yhat_proba)
    roc_auc = roc_auc_score(y, yhat_proba)
    
    return pd.DataFrame({ 'Model Name': model_name, 
                           'Precision': precision, 
                           'Recall': recall,
                           'F1 Score': f1,
                           'Kappa': kappa,
                           'Balanced Accuracy': balanced_accuracy,
                           'Average Precision Score': ap,
                           'ROC AUC': roc_auc}, index=[0])



ml_scores('Lightgbm', df_test_dataset['Exited'], d1['final_result'], d1['yhat_proba_level_02'])

,Model Name,Precision,Recall,F1 Score,Kappa,Balanced Accuracy,Average Precision Score,ROC AUC
0,Lightgbm,0.372289,0.759214,0.499596,0.311603,0.716079,0.579378,0.793519
